In [40]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [41]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [42]:
model = Sequential()

In [43]:
width = 1
height = 15
classes = 12

In [44]:
shape = (width, height, 1)

In [45]:
model.add(Conv2D(32, (3, 1), padding="same", input_shape=shape))

In [46]:
model.add(Activation("relu"))

In [47]:
model.add(BatchNormalization())

In [48]:
model.add(AveragePooling2D(pool_size=(1,2)))

In [49]:
model.add(Dropout(0.05))

In [50]:
model.add(Conv2D(64, (5, 1), padding="same"))

In [51]:
model.add(Activation("relu"))

In [52]:
model.add(BatchNormalization())

In [53]:
model.add(AveragePooling2D(pool_size=(1,2)))

In [54]:
model.add(Dropout(0.05))

In [55]:
model.add(Conv2D(128, (5, 1), padding="same"))

In [56]:
model.add(Activation("relu"))

In [57]:
model.add(AveragePooling2D(pool_size=(1,2)))

In [58]:
model.add(BatchNormalization())

In [59]:
model.add(Dropout(0.05))

In [60]:
model.add(Conv2D(256, (5, 1), padding="same"))

In [61]:
model.add(Activation("relu"))

In [62]:
model.add(BatchNormalization())

In [63]:
model.add(Dropout(0.05))

In [64]:
model.add(Flatten())

In [65]:
model.add(Dense(512))

In [66]:
model.add(Activation("relu"))

In [67]:
model.add(Dense(classes))

In [68]:
model.add(Activation("softmax"))

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 1, 15, 32)         128       
                                                                 
 activation_6 (Activation)   (None, 1, 15, 32)         0         
                                                                 
 batch_normalization_4 (Batc  (None, 1, 15, 32)        128       
 hNormalization)                                                 
                                                                 
 average_pooling2d_3 (Averag  (None, 1, 7, 32)         0         
 ePooling2D)                                                     
                                                                 
 dropout_4 (Dropout)         (None, 1, 7, 32)          0         
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 7, 64)         

### Data processing

In [70]:
import pandas as pd
import numpy as np

In [71]:
df = pd.read_csv("/content/drive/MyDrive/Deep Learning/iot23_malconcat_v2.csv", low_memory=False)

In [72]:
df.head()

,Unnamed: 0,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0,1360,60905.0,127613,23.0,1,0,2.998796,0.0,0.0,6,17,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan
1,1,1360,44301.0,1061393,23.0,1,0,0.000000,0.0,0.0,6,17,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan
2,2,1360,50244.0,83457,23.0,1,0,0.000000,0.0,0.0,6,17,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan
3,3,1360,34243.0,205149,49560.0,1,0,2.998804,0.0,0.0,6,17,3.0,180.0,0.0,0.0,Benign
4,4,1360,34840.0,195147,21288.0,1,0,0.000000,0.0,0.0,6,17,1.0,60.0,0.0,0.0,Benign


In [73]:
df = df.drop("Unnamed: 0", axis=1)

In [74]:
df.head()

,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1360,60905.0,127613,23.0,1,0,2.998796,0.0,0.0,6,17,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan
1,1360,44301.0,1061393,23.0,1,0,0.000000,0.0,0.0,6,17,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan
2,1360,50244.0,83457,23.0,1,0,0.000000,0.0,0.0,6,17,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan
3,1360,34243.0,205149,49560.0,1,0,2.998804,0.0,0.0,6,17,3.0,180.0,0.0,0.0,Benign
4,1360,34840.0,195147,21288.0,1,0,0.000000,0.0,0.0,6,17,1.0,60.0,0.0,0.0,Benign


In [75]:
#df = df.drop(columns=['id.resp_h', 'id.orig_h', 'id.orig_p', 'id.resp_p'])

In [76]:
list(df.columns)

['id.orig_h',
 'id.orig_p',
 'id.resp_h',
 'id.resp_p',
 'proto',
 'service',
 'duration',
 'orig_bytes',
 'resp_bytes',
 'conn_state',
 'history',
 'orig_pkts',
 'orig_ip_bytes',
 'resp_pkts',
 'resp_ip_bytes',
 'label']

In [77]:
df.fillna(-1, inplace=True)

In [78]:
from sklearn.preprocessing import StandardScaler

In [79]:
sc = StandardScaler()

X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

X[:,:] = sc.fit_transform(X[:,:])

In [80]:
X.shape

(1444674, 15)

In [81]:
Y.shape

(1444674,)

In [82]:
Y

array(['PartOfAHorizontalPortScan', 'PartOfAHorizontalPortScan',
       'PartOfAHorizontalPortScan', ..., 'DDoS', 'DDoS', 'DDoS'],
      dtype=object)

In [83]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

In [84]:
Y = lb.fit_transform(Y)

In [85]:
Y.shape

(1444674, 12)

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_vt, y_train, y_vt = train_test_split(X,Y, train_size=0.8, test_size=0.2, random_state=0)

In [87]:
X_test, X_valid, y_test, y_valid = train_test_split(X_vt,y_vt, train_size=0.5, test_size=0.5, random_state=0)

In [88]:
X_test.shape[1]

15

In [89]:
X_valid.shape[1]

15

In [90]:
X_train.shape

(1155739, 15)

In [91]:
X_train = X_train.reshape((X_train.shape[0],1,15,1))

In [92]:
X_train.shape

(1155739, 1, 15, 1)

In [93]:
X_valid.shape

(144468, 15)

In [94]:
X_valid = X_valid.reshape((X_valid.shape[0],1,15,1))

In [95]:
X_test = X_test.reshape((X_test.shape[0],1,15,1))

In [96]:
X_train.dtype

dtype('float64')

In [97]:
y_train.dtype

dtype('int64')

### Train model

In [98]:
learning_rate = 0.01

In [99]:
epochs = 10
batch_size = 64

In [100]:
opt = SGD(learning_rate=learning_rate, momentum=0.9)

In [101]:
#aug = ImageDataGenerator(rotation_range=0.18, zoom_range=0.15, height_shift_range=0.2, horizontal_flip=True)

In [102]:
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [103]:
print("Start training...")
H = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, verbose=1)
#H = model.fit(aug.flow(X_train, y_train, batch_size=batch_size), validation_data=(X_valid, y_valid), steps_per_epoch=X_train.shape[0]//batch_size, epochs=epochs, verbose=1)

Start training...
Epoch 1/10
36117/36117 [==============================] - 198s 5ms/step - loss: 0.2222 - accuracy: 0.9177 - val_loss: 0.3121 - val_accuracy: 0.8919
Epoch 2/10
36117/36117 [==============================] - 187s 5ms/step - loss: 0.1310 - accuracy: 0.9517 - val_loss: 0.2218 - val_accuracy: 0.9307
Epoch 3/10
36117/36117 [==============================] - 196s 5ms/step - loss: 0.1147 - accuracy: 0.9577 - val_loss: 0.1315 - val_accuracy: 0.9563
Epoch 4/10
36117/36117 [==============================] - 196s 5ms/step - loss: 0.1064 - accuracy: 0.9608 - val_loss: 0.2228 - val_accuracy: 0.9292
Epoch 5/10
36117/36117 [==============================] - 196s 5ms/step - loss: 0.0978 - accuracy: 0.9638 - val_loss: 0.1297 - val_accuracy: 0.9601
Epoch 6/10
36117/36117 [==============================] - 195s 5ms/step - loss: 0.0944 - accuracy: 0.9651 - val_loss: 4.1974 - val_accuracy: 0.2140
Epoch 7/10
36117/36117 [==============================] - 198s 5ms/step - loss: 0.0906 - accur

### Test Model

In [106]:
import numpy as np

In [107]:
result = model.predict(X_test[2000:2001])
final = np.argmax(result)
final

11

In [108]:
np.argmax(y_test[2000])

11